E

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

from nltk.stem.snowball import SnowballStemmer
import re

data = pd.read_csv("arxiv_data/arxiv_dataset.csv", index_col=0)

X_train = data.loc[:2999, 'sampled_sentence']
X_test = data.loc[3000:, 'sampled_sentence']
y_train = data.loc[:2999, 'paper_section']

# Инициализируем стеммер
stemmer = SnowballStemmer("english")

# Кастомная токенизация + стемминг
def tokenize_and_stem(text):
    # Базовая нормализация текста
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.strip().split()
    return [stemmer.stem(token) for token in tokens if len(token) > 2]

vectorizer = TfidfVectorizer(
    tokenizer=tokenize_and_stem,
    ngram_range=(1, 2),          # unigrams + bigrams
    stop_words='english',        # убираем частые слова
    max_df=0.06,                 # убираем супервстречающиеся слова
)

X_train_vec = vectorizer.fit_transform(X_train)  # sparse matrix
X_test_vec = vectorizer.transform(X_test)

model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)

📌 1. Извлечение признаков (Feature Extraction)
📍 CountVectorizer (для текстов)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

texts = ["I love cats", "Cats are lovely"]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

print(vectorizer.get_feature_names_out())  # ['are' 'cats' 'love' 'lovely']
print(X.toarray())  # [[0 1 1 0], [1 1 0 1]]


📍 TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

texts = ["I love cats", "Cats are lovely"]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)

print(X.toarray())


📍 PCA (уменьшение размерности)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris

X = load_iris().data
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

print(X_reduced[:5])  # Сокращённые признаки (2 признака)


📍 groupby в pandas

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'user': ['A', 'A', 'B', 'B'],
    'score': [10, 20, 30, 40]
})

agg = df.groupby('user')['score'].mean().reset_index()
print(agg)  # Средний score по user


Вывод:
  store  day  sales
0     A  Mon    100
1     A  Tue    150
2     B  Mon    200
3     B  Tue    180
4     B  Wed    160
5     C  Mon     90


📌 Цель: найти среднюю выручку по каждому магазину

In [ ]:
# Группируем по магазину и считаем среднюю выручку
avg_sales = data.groupby('store')['sales'].mean().reset_index()
avg_sales.columns = ['store', 'avg_sales']

print(avg_sales)


  store  avg_sales
0     A      125.0
1     B      180.0
2     C       90.0


🔁 Ещё пример: общая выручка по каждому магазину и дню

In [ ]:
total = data.groupby(['store', 'day'])['sales'].sum().reset_index()
print(total)

  store  day  sales
0     A  Mon    100
1     A  Tue    150
2     B  Mon    200
3     B  Tue    180
4     B  Wed    160
5     C  Mon     90


Отлично! Ниже покажу 2 мощных приёма с `groupby` — для **feature engineering** в ML-задачах:

---

## ✅ 1. `groupby + transform`: создаём фичу "отклонение от среднего в группе"

Допустим, у нас есть заказы клиентов в разных городах:

```python
import pandas as pd

df = pd.DataFrame({
    'city': ['A', 'A', 'B', 'B', 'C', 'C'],
    'user_id': [1, 2, 3, 4, 5, 6],
    'order_amount': [100, 150, 200, 250, 80, 120]
})
```

---

### 📌 Цель: для каждой строки найти **отклонение от среднего по городу**

```python
# Считаем средний чек по городу
df['city_avg'] = df.groupby('city')['order_amount'].transform('mean')

# Вычисляем отклонение от среднего
df['dev_from_city_avg'] = df['order_amount'] - df['city_avg']

print(df)
```

**Вывод:**

```
  city  user_id  order_amount  city_avg  dev_from_city_avg
0    A        1           100     125.0              -25.0
1    A        2           150     125.0               25.0
2    B        3           200     225.0              -25.0
3    B        4           250     225.0               25.0
4    C        5            80     100.0              -20.0
5    C        6           120     100.0               20.0
```

👉 Это полезно, если ты хочешь учитывать поведение пользователя относительно окружения (например, выше или ниже среднего по группе).

---

## ✅ 2. `groupby + apply`: создаём сложную фичу — ранжирование внутри группы

```python
# Присваиваем ранги пользователям по сумме заказов внутри каждого города
df['rank_in_city'] = df.groupby('city')['order_amount'].apply(lambda x: x.rank(ascending=False))

print(df)
```

**Вывод:**

```
  city  user_id  order_amount  rank_in_city
0    A        1           100           2.0
1    A        2           150           1.0
2    B        3           200           2.0
3    B        4           250           1.0
4    C        5            80           2.0
5    C        6           120           1.0
```

📍 Используется, например, в рекомендациях или кредитном скоринге: кто лучший в своей категории?

---

Хочешь, покажу, как эти фичи можно потом передать в модель (например, LightGBM)?


In [ ]:
import pandas as pd

# Пример датафрейма
data = pd.DataFrame({
    'store': ['A', 'A', 'B', 'B', 'B', 'C'],
    'day': ['Mon', 'Tue', 'Mon', 'Tue', 'Wed', 'Mon'],
    'sales': [100, 150, 200, 180, 160, 90]
})

print(data)


📍 Градиентный бустинг (на табличных данных)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=100, n_features=4)
model = GradientBoostingClassifier()
model.fit(X, y)

print(model.predict(X[:5]))


📍 KFold кросс-валидация

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

X = np.arange(10)
kf = KFold(n_splits=5)

for train_index, val_index in kf.split(X):
    print("Train:", train_index, "Val:", val_index)
